# util

> Collection of various more advanced single-cell plots

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

from fastcore.test import *
from fastcore.docments import docments

import numpy as np
from numpy.random import default_rng

from scipy.sparse.csgraph import connected_components

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.patches import ConnectionPatch

We are primarily going to be working with non-model species, so the gene names will always be of the form

`XLOC_123456 | emapper-name-or-description-if-we're-lucky`

or something similar. This means that we could have extreme variation in the actual length of a gene "name"; this will make it very hard to put gene names on axes as it will distort figure sizes. I wrote a function to either trim or pad strings; even though axis labels are not in monospace fonts, it is much easier to visually reconcile strings with lengths in the same order of magnitude.

In [ ]:
#| export
def procrustes(x:str, # input string
               appropriate_length:int=50, # desired length
               pad_with:str=" ", # character to pad with
               side:str="right" # which side to pad on ("left", "right")
              )->str: # string with desired length
    "A function to regulate string length."
    if len(x) > appropriate_length:
        return x[:appropriate_length]
    if len(x) < appropriate_length:
        to_pad = appropriate_length - len(x)
        pad = "".join([pad_with] * to_pad)
        if side == "right":
            x = x + pad
        elif side == "left":
            x = pad + x
        else:
            print("Invalid side argument; returning string as-is.")
    return x

In [ ]:
too_short = 'Niko'
just_right = 'Theseus'
too_tall = 'The Mountain'

In [ ]:
assert procrustes(just_right, appropriate_length=7) == 'Theseus'
assert procrustes(too_short, appropriate_length=7) == 'Niko   '
assert procrustes(too_tall, appropriate_length=7) == 'The Mou'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()